In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
#hyper parameters
num_epochs=10
num_classes=10
batch_size=4
learning_rate=0.001
input_size = 1024

In [11]:
#dataset has PIL images of size [0,1]
#transform them to tensors of range [-1,1]

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [12]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
examples = iter(train_loader)
samples, labels = examples.next()
print(samples.shape, labels.shape)


torch.Size([4, 3, 32, 32]) torch.Size([4])


In [10]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [11]:
#implement convnet

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)         6,28,28
        self.pool = nn.MaxPool2d(2, 2)          6,14,14
        self.conv2 = nn.Conv2d(6, 16, 5)        16,10,10 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net()

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [13]:
#training
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        output = model(images)
        loss = criterion(output, labels)
        
        #backward and optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            
print('Finished Training')
        
    
    
    

Epoch [1/10], Step [2000/12500], Loss: 2.2475
Epoch [1/10], Step [4000/12500], Loss: 2.2995
Epoch [1/10], Step [6000/12500], Loss: 2.3064
Epoch [1/10], Step [8000/12500], Loss: 2.2599
Epoch [1/10], Step [10000/12500], Loss: 1.9617
Epoch [1/10], Step [12000/12500], Loss: 1.7477
Epoch [2/10], Step [2000/12500], Loss: 1.7268
Epoch [2/10], Step [4000/12500], Loss: 2.2024
Epoch [2/10], Step [6000/12500], Loss: 1.8493
Epoch [2/10], Step [8000/12500], Loss: 1.9940
Epoch [2/10], Step [10000/12500], Loss: 2.4617
Epoch [2/10], Step [12000/12500], Loss: 1.3883
Epoch [3/10], Step [2000/12500], Loss: 1.7538
Epoch [3/10], Step [4000/12500], Loss: 2.0058
Epoch [3/10], Step [6000/12500], Loss: 1.1916
Epoch [3/10], Step [8000/12500], Loss: 1.5498
Epoch [3/10], Step [10000/12500], Loss: 1.2519
Epoch [3/10], Step [12000/12500], Loss: 1.5357
Epoch [4/10], Step [2000/12500], Loss: 1.6608
Epoch [4/10], Step [4000/12500], Loss: 0.9803
Epoch [4/10], Step [6000/12500], Loss: 1.5645
Epoch [4/10], Step [8000/125

In [21]:
#testing

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 56.03 %
Accuracy of plane: 61.1 %
Accuracy of car: 76.2 %
Accuracy of bird: 41.9 %
Accuracy of cat: 30.3 %
Accuracy of deer: 36.5 %
Accuracy of dog: 38.4 %
Accuracy of frog: 71.4 %
Accuracy of horse: 71.8 %
Accuracy of ship: 66.4 %
Accuracy of truck: 66.3 %
